In [12]:
import cv2
import numpy as np
import subprocess
from PIL import Image, ImageDraw, ImageFont

# Paths
orig_path = "C:/Users/27118/Desktop/UCL/master_project/Appendix/pics2/StMarysChurch_orig.png"
render_path = "C:/Users/27118/Desktop/UCL/master_project/Appendix/pics2/StMarysChurch_loc.png"

# === Step 1: Load images ===
orig = cv2.imread(orig_path)
render = cv2.imread(render_path)
h, w = orig.shape[:2]
render = cv2.resize(render, (w, h))

# === Step 2: Add labels ===
from PIL import ImageFont

font = ImageFont.truetype("C:/Windows/Fonts/times.ttf", 32)


def draw_label(img_cv, text, position, color):
    img_pil = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

orig_labeled = draw_label(orig, "query image", (10, 10), (0, 0, 255))
tw, th = font.getsize("rendered with estimated pose")
render_labeled = draw_label(render, "rendered with estimated pose", (w - tw - 10, h - th - 10), (0, 255, 0))

# === Step 3: Create transition frames ===
orig_f = orig_labeled.astype(np.float32) / 255.0
render_f = render_labeled.astype(np.float32) / 255.0

frames = []
num_frames = 120
pause_frames = 45
line_thickness = 4

def draw_transition(t, forward=True):
    threshold = (t / (num_frames - 1)) * (w + h) if forward else (1 - t / (num_frames - 1)) * (w + h)
    alpha = np.zeros((h, w), np.float32)
    for y in range(h):
        for x in range(w):
            if x + y < threshold:
                alpha[y, x] = 1.0
    mask = np.stack([alpha]*3, axis=-1)
    blended = (mask * render_f + (1 - mask) * orig_f) * 255
    blended = blended.astype(np.uint8)

    for y in range(h):
        x = int(threshold - y)
        if 0 <= x < w:
            for dx in range(-line_thickness//2, line_thickness//2+1):
                if 0 <= x+dx < w:
                    blended[y, x+dx] = (255,255,255)

    for i in range(line_thickness):
        for x in range(w):
            if 0 <= i < h:
                blended[i, x] = (0,255,0) if x+i < threshold else (255,0,0)
                blended[h-1-i, x] = (0,255,0) if x+(h-1-i) < threshold else (255,0,0)
        for y in range(h):
            if 0 <= i < w:
                blended[y, i] = (0,255,0) if i+y < threshold else (255,0,0)
                blended[y, w-1-i] = (0,255,0) if (w-1-i)+y < threshold else (255,0,0)

    return blended

for t in range(num_frames):
    frames.append(draw_transition(t, forward=True))

frames += [frames[-1]] * pause_frames

for t in range(num_frames):
    frames.append(draw_transition(t, forward=False))

frames += [frames[-1]] * pause_frames

# === Step 4: Save raw video ===
raw_path = "C:/Users/27118/Desktop/logs.github.io/assets/StMarysChurch_transition_raw.mp4"
out = cv2.VideoWriter(raw_path, cv2.VideoWriter_fourcc(*"mp4v"), 30, (w, h))
for f in frames:
    out.write(f)
out.release()

raw_path


C:\Users\27118\AppData\Local\Temp\ipykernel_54492\3008078305.py:29: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  tw, th = font.getsize("rendered with estimated pose")


'C:/Users/27118/Desktop/logs.github.io/assets/StMarysChurch_transition_raw.mp4'

In [13]:
# Step 5: Convert to H.264 encoded video
h264_path = "C:/Users/27118/Desktop/logs.github.io/assets/StMarysChurch_transition_h264.mp4"
subprocess.run([
    "ffmpeg", "-y", "-i", "C:/Users/27118/Desktop/logs.github.io/assets/StMarysChurch_transition_raw.mp4",
    "-vcodec", "libx264", "-preset", "slow",
    "-crf", "18", "-pix_fmt", "yuv420p",
    "-movflags", "+faststart", h264_path
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

h264_path


'C:/Users/27118/Desktop/logs.github.io/assets/StMarysChurch_transition_h264.mp4'

In [14]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path

# === 配置 ===
image_dir = Path("C:/Users/27118/Desktop/logs.github.io/assets")
font_path = "C:/Windows/Fonts/times.ttf"  # 使用 Times 字体
font_size = 32
font = ImageFont.truetype(font_path, font_size)
output_suffix = "_label"

# === 目标文件名 ===
image_names = [
    "fire_compare.png", "heads_compare.png", "stairs_compare.png",
    "pumpkin_compare.png", "chess_compare.png", "redkitchen_compare.png",
    "office_compare.png", "KingsCollege_compare.png",
    "ShopFacade_compare.png", "StMarysChurch_compare.png"
]

# === 标注函数 ===
def draw_label(img_cv, text, position, color):
    img_pil = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 开始处理 ===
for name in image_names:
    path = image_dir / name
    img_cv = cv2.imread(str(path))
    if img_cv is None:
        print(f"读取失败：{name}")
        continue

    h, w = img_cv.shape[:2]
    tw1, th1 = font.getbbox("rendered with estimated pose")[2:]
    tw2, th2 = font.getbbox("query image")[2:]

    img_cv = draw_label(img_cv, "rendered with estimated pose", (w - tw1 - 10, 10), (0, 255, 0))
    img_cv = draw_label(img_cv, "query image", (10, h - th2 - 10), (0, 0, 255))

    output_path = image_dir / f"{Path(name).stem}{output_suffix}.png"
    cv2.imwrite(str(output_path), img_cv)

print("✅ 全部图像标注完成。")


✅ 全部图像标注完成。
